In [ ]:
import numpy as np
import torch 
import einops
import matplotlib.pyplot as plt
import torch.nn as nn
from scipy import stats

Throughout this notebook we assume that the vectors are unit norm and that they are trained using some kind of contrastive loss. Unit norm forces the model to represent every feature in the output space, i.e. it cannot cop out by turning things off. While contrastive loss for search, allows us to interpret the geometry. As opposed to MLM or Auto-regressive loss where the geometry of the final layer is not necessarily constrained by any geometry; search transformers with contrastive loss literally demands similar vectors to be closer to each other and vice versa. Thus the objective of contrastive loss for search transformer allows us to study the geometry of the model output without us running into circles.

Let us discuss each of the above three points with some code snippets and sample experiments. The following is an attempt to create a toy model for the data distribution and the model that will hopefully allow us to answer some questions about vector embeddings progressively. 

# 1) Geometric intuition for superposition and its relation to isotropy of vector embeddings 

Let us imagine a 2-dimensional vector space for the moment, and assume that the data is sampled from a latent uniform discrete distribution $P(z)$, where $z \in \{1,2,3,5, \ldots k\}$. We refer to $k$ as the vocab size in this notebook. 

We can think of the variable $z$ as taking values in the space of semantic meanings. So perhaps 1 = “Fruit”, 2 = “Physics”, 3 = “Batman”, 4 = “Red” and so on.  We want to learn 2-dimensional unit vector representations V(z) such that, 



$$ V(z = i) \cdot V(z = j) = 2*\delta_{ij} - 1 \tag{1}$$ 

This requirement is nothing but the contrastive loss used for training vector embeddings for sentence similarity/search. Although this requirement is certainly not realistic, since not every semantic concept is unrealted to another. We will relax this assumption soon. Recall that we work with unit norm vectors throughout this notebook. 

Given the above simplisitc setup and the fact that P(z) is uniformly distributed, we can show that the optimal vectors that we could learn  would be the $k$ roots of unity (up to permutations). 


**If we were in $k$ dimensions the optimal assignment would be one hot encoding for each vector $V(z = i)$. But since we are in fewer dimensions, the two cardinal directions are in superposition and the $k$ roots of unity is the best compromise.** We can also check this by runniing a few quick experiments

In [ ]:
class FCC(nn.Module):
    def __init__(self, vocab_size, hidden_dim : list, output_dim) -> None:
        super().__init__()
        dims = hidden_dim + [output_dim]
        self.V = vocab_size        
        self.layers = nn.ModuleList()
        self.reslayers = nn.ModuleList()
        self.relu = nn.ReLU()
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(self.V,dims[0])
        
        for i in range(len(dims)-2):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.reslayers.append(nn.Linear(dims[i],dims[i+1]))
        self.layers.append(nn.Linear(dims[-2],dims[-1]))
    
    def forward(self,x):
        x = self.embed(x)        
        
        for layer, reslayer  in zip(self.layers[:-1], self.reslayers):
            y = layer(x)
            x = self.relu(y) + reslayer(x)                    
        x = self.layers[-1](x) 
        x = x / torch.norm(x,dim=-1,keepdim=True)
        
        return x
    

    def train(self,x,epochs,lr,batch_size):
        optimizer = torch.optim.Adam(list(self.layers.parameters()) + list(self.reslayers.parameters()) + list(self.embed.parameters()), lr=lr)
        for e in range(epochs):
            perm = torch.randperm(len(x))
            x = x[perm]            
            labels = (einops.rearrange(x, " i -> 1 i") == einops.rearrange(x, " i -> i 1")).float()
            labels = 2*labels - 1
            
            for b in range(0,len(x),batch_size):
                output = self.forward(x[b:b+batch_size].to(device))
                targets = labels[b:b+batch_size,b:b+batch_size].to(device)                                
                out = output @ output.T
                optimizer.zero_grad()  
                loss = torch.mean((targets - out)**2)
                loss.backward()
                optimizer.step()

            print(loss.item())
            if loss.item() <= optimal_loss*1.01:
                print(f'Almost hit optimal loss. Early stopping.')
                return 
                
            with torch.inference_mode():
                if e % 2 == 0:
                    out_vec = self.forward(torch.arange(self.vocab_size,device=device)).cpu().numpy()                                        
                    theta = np.linspace(0, 2*np.pi, 100);  cx = np.cos(theta); cy = np.sin(theta); fig, ax = plt.subplots(); ax.plot(cx, cy); ax.set_aspect('equal')
                    plt.scatter(np.hstack((np.array(0),out_vec[:,0])), np.hstack((np.array(0),out_vec[:,1])) )
                    plt.savefig("out_vecs")                
                    plt.show()
                    
def make_data(samples, concepts):        
    inp = einops.repeat(torch.arange(0,concepts), " i -> (i b)", b = samples//concepts)
    inp = inp[torch.randperm(len(inp))]
    return inp

In [ ]:
device = "cpu"
k = 5

In [ ]:
# This cell computes the loss when the vectors take the shape of k'th roots of unity
gt = [1] + [-1]*(k-1)
diffs = (gt - np.cos((np.pi/180)*np.linspace(0,360,k+1)[:-1]))
optimal_loss = np.mean(diffs**2)
print(f'The optimal loss for vocab size {k} is {optimal_loss}')

In [ ]:
lr = 5e-2
batch_size = 2048*4    
netter = FCC(k,[k//2,2],2)
netter.to(device)

In [ ]:
train = make_data(50_000,k)
netter.train(train,epochs = 100,lr = lr,batch_size = batch_size)

**I have obtained vector assignments as roots of unity upto 7. But with larger $k$, I find it harder (although I haven't tried that much) to get roots of unity. Also sometimes, even for small $k$ (especially when its an even number), it tries to cheat by overlapping the vectors. There is probably a degeneracy for even $k$.** Note that we use an MSE loss for training. It is possible that for cross entropy loss there might be better ways to hedge as was found in the SAE dictionary learning paper by Anthropic.

There are two possible immediate questions of interest. Both are guided by the desire to understand how real transformers behave when trained on the task of sentence similarity. 

But before we dive further, let me establish the analogy of actual transformers with our toy model. In our model we are trying to come up with vector assignments, in real life (and in more complex examples that follow) the transformer/network does this using gradient descent.  In our model, the goal is given by equation (1), while for the real transformer it is given by some contrastive loss. Finally, in real life the data points are sentences that come from the empirical distribution of natural language on the internet. While here a data point is the discrete variable $z$ which is obtained by sampling from some $P(z)$. 

## 1.1)  What if the distribution is not uniform (but say Zipfian)? 

Let us say that $P(z)$ is Zipfian where $z=1$ is most frequent and $z = k$ is least frequent. What would be the optimal assignment? Let us think about it iteratively. 

If $k=2$, the solution remains the same – roots of unity. But as $k$ increases, subsequent $V(z)$ for $z > 2$, will try to stay close to $V(z=2)$ to minimize the contrastive loss. This is simply because all V(z) want to stay antipodal to each other but there is no room left after the first two vectors $V(z = 1)$ and $V(z = 2)$ are assigned antipodal points. Thus the subsequent vectors will try to cut their losses and stay close to $V(z = 2)$ to minimize the loss on the empirical distribution as there are way many more $V(z = 1)$ than there are $V(z = 2)$. 

This, of course, can only stay true up to some point, after which there is no more room left and $V(z = i)$ for large $i$ will have to start getting closer to $V(z_1)$. The question is to understand how and when this happens for arbitrary dimensions $d$ and for more interesting real life distributions. Let us try to run some experiments. The set up remains the same as before, except that our make_data function needs to be changed




In [ ]:
def make_zipf_data(most_frequent_size, concepts):        
    tensor_list = []
    for i in range(1,concepts+1):
        samples_i = most_frequent_size // i 
        inp = einops.repeat(torch.tensor([i-1]), " i -> (i b)", b = samples_i)
        tensor_list.append(inp)
    inp = torch.cat(tensor_list)
    inp = inp[torch.randperm(len(inp))]
    return inp

### For $k=2$ we find expected behavior -- antipodal points.

In [ ]:
device = "cpu"
k = 2
optimal_loss = 1e-8

In [ ]:
lr = 5e-2
batch_size = 2048*4    
netter = FCC(k,[2],2)
netter.to(device)
train = make_zipf_data(10000,k)
netter.train(train,epochs = 20,lr = lr,batch_size = batch_size)

### What happens if we now change $k$ to 3. 

It is not easy to derive the optimal loss for $k=3$ analytically but we can calculate it numerically. For the Zipf distribution (with exponent 1), z="0" appears 6/11 times, z="1" appeards 3/11 times and z="2" appears 2/11 times. The average loss is given by,
$$ \frac{2}{121}\left( 18 (-1-\cos \theta_1)^2 + 12 (-1-\cos \theta_2)^2 +  6 (-1-\cos (|\theta_1 - \theta_2|))^2 \right) $$







Notation: The vector "0" lies on the positive y-axis WLOG. The vector "1" makes an angle of $\theta_1$ with it, and the vector "2" makes an angle of $\theta_2$. The angle between vector "1" and "2" is thus given by $|\theta_1 - \theta_2|$.

In [ ]:
# Numerical simulation

optimal_loss = 1e10
for x1 in np.arange(0,2*np.pi,0.01):
    for x2 in np.arange(0,2*np.pi,0.01):
        loss = (2/121)*(18*(-1-np.cos(x1))**2 + 12*(-1-np.cos(x2))**2+ 6*(-1-np.cos(np.abs(x1-x2)))**2)
        if loss < optimal_loss:
            optimal_loss = loss
            angles = [x1,x2]

The numerical simulation gives us the value of the optimal loss and the values for $\theta_1$ and $\theta_2$

In [ ]:
optimal_loss, angles[0]*(180/np.pi), angles[1]*(180/np.pi)
# angles[0]*(180/np.pi)-180, 180-angles[1]*(180/np.pi)

### That is indeed what we find with gradient descent as shown below

In [ ]:
device = "cpu"
k = 3

In [ ]:
lr = 5e-2
netter = FCC(k,[2],2)
netter.to(device)
train = make_zipf_data(20000,k)
netter.train(train,epochs = 100,lr = lr,batch_size = len(train))

### What if the zipf law had an exponent smaller than 1, i.e. the drop in frequency is less precipitious?

The loss for this case will be larger and is given by (for exponent = 0.5),

In [ ]:
optimal_loss = 1e10
surd = np.sqrt(6) + np.sqrt(3) + np.sqrt(2)
for x1 in np.arange(0,2*np.pi,0.01):
    for x2 in np.arange(0,2*np.pi,0.01):
        loss = (2/surd**2)*(np.sqrt(18)*(-1-np.cos(x1))**2 + np.sqrt(12)*(-1-np.cos(x2))**2+ np.sqrt(6)*(-1-np.cos(np.abs(x1-x2)))**2)
        if loss < optimal_loss:
            optimal_loss = loss
            angles = [x1,x2]

In [ ]:
optimal_loss, angles[0]*(180/np.pi), angles[1]*(180/np.pi)
# 180 - angles[0]*(180/np.pi), angles[1]*(180/np.pi) - 180

The angles increase when compared to Zipf with exponent = 1. This makes sense. As the exponent gets smaller, the distribution gets closer to being a uniform distribution, and the angles start approaching 60 (i.e. cubic roots of unity).

Following is the code that runs the experiment when the Zipf exponent is 0.5.

In [ ]:
def make_zipf_data_with_exp(most_frequent_size, concepts,exp = 0.5):        
    tensor_list = []
    for i in range(1,concepts+1):
        samples_i = int(most_frequent_size / (i**exp))
        inp = einops.repeat(torch.tensor([i-1]), " i -> (i j)", j = int(samples_i))
        tensor_list.append(inp)
    inp = torch.cat(tensor_list)
    inp = inp[torch.randperm(len(inp))]
    return inp

In [ ]:
k = 3

In [ ]:
lr = 1e-2
netter = FCC(k,[3,2],2)
netter.to(device)
train = make_zipf_data_with_exp(20000,k,exp=0.5)
netter.train(train,epochs = 100,lr = lr,batch_size = len(train))

These set of preliminary experiments were designed to study the optimization of the feature space in a controlled setting. We want to study larger space dimensions and different probability distributions. Increasing the space dimensions, definitely increases the expressivity and gets increasingly harder to analyse. I do not have a good systemtic way of analysing that, yet.

But even if we increase the space dimensions, this model still remains a toy model. That is because the probability distribution that we study is extremely simple. Even if we increase $k$ arbitrarily, it is not at all representative of natural language. And this brings us to the second question.

## 1.2) What if the data is obtained by sampling from a multivariate distribution $P(\vec{z})$?

Let us not worry about the actual distribution right now, except that it’s multivariate and that $\vec{z}$ is two dimensional. Using the previous example 1 = “Fruit”, 2 = “Physics”, 3 = “Batman”, 4 = “Red”, a sentence about “Apples” is now labeled by the discrete vector $\vec{z} = (1, 4)$, i.e. a Red Fruit.  

We are interested in multivariate distribution since at the end of the day we want to model actual sentences that are sampled empirically from the internet. And just using a univariate distribution to encapsulate the distribution of all sentences is obviously very limiting. 

The first thing we need is an analogue of equation (1) to proceed further. One possible candidate is, 

$$
\begin{align}
V(z_1,w_1) \cdot V(z_2,w_2) & = 1, \quad \text{if} \quad z_1 = z_2 \, \text{and} \, w_1 = w_2 \\
V(z_1,w_1) \cdot V(z_2,w_2) & = 0, \quad \text{if} \quad z_1 = z_2 \,  \text{or} \, w_1 = w_2 \, \text{but not both} \\
V(z_1,w_1) \cdot V(z_2,w_2) & = -1, \quad \text{if} \quad z_1 \neq z_ 2 \, \text{and} \,  w_1 \neq  w_2 
\end{align} \tag{2}
$$

Although this is far from ideal, it does capture some key aspects of similarity. For instance, Apple (red, fruit) and Banana (yellow, fruit) will have similarity of 0, while Apple (red,fruit) and Einstein (theoretical, physicist) will have similarity of -1. Of course there are limitations. Since this metric will also give a similarity of 0 for Apple (red, fruit) and Newton (theoretical, physicist) which would be technically correct but wrong for reasons of folklore ;) 

Now that we have the model ready, what is the optimal assignment of vectors? I do not know the answer to this question. But my hope is that in answering this question for our toy model, we will understand what features are relevant and selected by the neural network during gradient descent, and how it all manifests in vector embeddings. The dream is to do this for arbitrary vector dimensions of V and for multivariate distributions with modestly large dimensions of $\vec{z}$. 


### Thesis for Point 1

In this part we have tried to build a framework to study vector assignments using constrastive loss. One of the reasons for studying this is because it is closely related to feature representations. In the wild, a transformer learns the semantic concepts ($z_i$) that are present in the training data and assigns them directions. Since the number of space dimensions are far smaller than the number of concepts, the model starts exhibiting superposition. The toy models and data distrbution (albeit unrealistic) presented in this section can help us study superposition systematically.

If you have read so far and wondering where is the mechanistic interpretability, I'm afraid I do not have anythig super concrete to offer. But I do have some ideas... 

1) Recent literature has shown that learning sparse representations has been hard -- i.e. it is possible to get only one neuron fire at a time, but it is hard to make that neuron monosemantic. This is related to the fact that the model can hedge minimizing cross entropy loss with polysemanticity. The toy problem proposed here can help us answer -- the order and rate at which neurons become polysemantic. In our Zipf section 1.1, we argued that less frequent features get superposed first. 

2) In the context of sparse autoencoders, the features that it learns are the $z_i$ and these are being encoded by the model in $d \ll k$ dimensions. Can we study this encoding using our toy model? Can our toy model shed light into what features are learnt and the strenght at which a neuron fires for a given feature? For small $k$ we can probably solve this analytically.

3) Conversely, how does the space dimensionality affect the number of features the model can represent? For instance, in our very first code example, gradient descent for our simple neural network was not able to find $k$the roots of unity when $k$ got larger than 7. In other words, a very simple sparse autoencoder (that only cares about relative orientation and roots of unity) used to analyse our toy neural network might not be able to find more than 7 features. 

4) Our analysis is for unit norm vectors, i.e. the model can not turn off anything. This changes the problem from the regular polytopes setting of Olah et al. **(I wish I was more familiar with the paper on toy models of superposition by his group before I started writing this idea! I fortunately/unfortunately realize now that there is a significant overlap)** where many features were just never expressed/mapped to the origin. Our model will thus have different dynamics and is important to study independetly since many search transformers are trained this way.


# 2) Optimality of gradient descent based transformer architectures 

Another question that comes up naturally here is do sufficently large transformer networks using SGD reach the optimal configuration?  For instance in section 1, a transformer with the right training should be able to find $k$th roots of unity for arbitrarty $k$ for uniform distrbution. Does that happen in practice?

What about Zipfian distributions with large physical dimensions $d$? For such distributions we do not have analytic control. So how do we even measure performance? Perhaps one way would be to use some kind of semidefinite programming (SDP).

The following code is an unsuccesful attempt, it does not work for $k > 3$. The issue is probably with triangle inequality: using SDP we can only create constraints that are linear in $x_i \cdot x_j$, but the triangle inequality requires a square root. On the other hand, we cannot impose triangle inequality on the squared distances, since squared distances do not satisfy the triangle inequality. Or may we don't need to worry about triangle inequality (since X is a gram matrix) and it is some other bug/issue. I haven't had the time to think about it more thoroughly.

Anyway, the idea is to find optimal configurations using SDP and then check if transformers can achieve that. The following code follows the notation from Wikipedia https://en.wikipedia.org/wiki/Semidefinite_programming#:~:text=%5Bedit%5D-,Initial%20motivation,-%5Bedit%5D 

The definition of $c$ below encodes the loss function and the underlying data distrbution. For uniform distribution
each entry is 1 or -1. But we can change these values to mimic other distributions. For intance for a Zipf distribution, the first row of $c$ would look like [-1,1/2,1/3,1/4, . . . ].

In [ ]:
import cvxpy as cp
k = 4
n = k
c = np.diag([-1]*n)
c[c != -1] = 1

A = []
b = []

# Impose unit norm on all vectors
for i in range(n):
    a = np.zeros((n,n))
    a[i,i] = 1
    A.append(a)
    b.append(1)
n1 = len(b)

# Impose upper bound on dot products to be 1    
for i in range(n):
    for j in range(n):
        a = np.zeros((n,n))
        a[i,j] = 1
        A.append(a)
        b.append(1)
n2 = len(b)        
        
# Impose lower bound on dot products to be -1           
for i in range(n):
    for j in range(n):
        a = np.zeros((n,n))
        a[i,j] = 1
        A.append(a)  
        b.append(-1)
n3 = len(b)

# # Impose triangle inequality
# for i in range(n):
#     for j in range(i+1,n):
#         for k in range(j+1,n):
#             a = np.zeros((n,n))
#             a[j,j] = 1
#             a[i,j] = -1
#             a[j,k] = -1
#             a[i,k] = 1
#             A.append(a)
#             b.append(0)            
# n4 = len(b)
            
X = cp.Variable((n, n), symmetric=True)
objective = cp.Minimize(cp.trace(c @ X))

# Cnstraint for semidefinite positive
constraints = [X >> 0]  

for i in range(n1):
    constraints.append(cp.trace(A[i] @ X) == b[i])
    
for i in range(n1,n2):
    constraints.append(cp.trace(A[i] @ X) <= b[i])
    
for i in range(n2,n3):    
    constraints.append(cp.trace(A[i] @ X) >= b[i])
    
# for i in range(n3,n4):    
#     constraints.append(cp.trace(A[i] @ X) >= b[i])    

prob = cp.Problem(objective, constraints)
prob.solve()

print("Optimal value and solution:")
print(prob.value,"\n", X.value)

### Thesis for point 2

In this short section we proposed that using a simple toy model for the data distribution, we can investigate how good are transformers at assigning directions to the concepts $z_i$. The optimal solution for simple distributions can be obtained analyitcally, while for others we could use something like SDP or other approximate methods (that we need to investigate further). If we could solve, we could ask a lot more interesting questions.

1. Is locality an obvious emergent property? We could use SDP to solve the problem in section 1.2. Perhaps the optimal solution **is** to map different kinds of birds together in a small neighborhood. This would then automatically also imply feature splitting for SAE. Since SAE with more parameters/resolution can zoom in further in these neighborhoods. 

2. If we remove the contraint that the vectors are unit norm for the SDP and the transformer, it opens up to another set of interesting questions. For instance, for non-normalized vectors, the optimal solution might be to map $m$ features to the origin and have the rest $k-m$ embedded isotropically on $S^{d-1}$ in $d$ physical dimensions (as shown by Anthropic in their work on regular polytopes). How does $m$ change as we change the parameters of the distribution (e.g. the exponent in Zipfian distributions)? 

3. Finally, these findings could help us examine the anistropy of vector embeddings that has has been found in several BERT like models. Which is what we turn to in the following final section.

# 3) Demystify anisotropy of vector embeddings

Enough talk, lets get real now. What happens with actual transformer embeddings. A lot of literature has been spent on studying anistropy of vector embeddings of BERT (starting with Ethayarajh https://aclanthology.org/D19-1006/). But these studies were for hidden vector states for MLM or Auto-regressive models. 

Given FP32 precision for models and the cross entropy objective it is not a priori necessary that hidden layer activations have to be isotropic or otherwise. Conversely, if hidden activations are anisotropic -- that does not necessarily mean that the model is less expressive or degenerate. Who knows what the model does; may be GPT-7 in its last layer is trying to trick its master by making everything super anisotropic! In other words, I am not sure if trying to decipher the geometry makes a whole lot of sense here. 

### Search transformers

But the same is not the case for Search transformers. They are trained with contrastive loss, keep relevant things close together and irrelevant things far apart -- it makes sense to study the geometry.

What happens when we study this geometry? We find a very good case of isotropy. Lets do a quick analysis. 

In [ ]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

dataset = "msmarco"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(pathlib.Path("__file__").parent.absolute(), "datasets")
data_path = util.download_and_unzip(url, out_dir)

corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="train")

In [ ]:
marco_pass = []
for k,v in qrels.items():
    for key in v.keys():
        marco_pass.append(corpus[key]["text"])

In [ ]:
# Passages from the popular MS Marco dataset
marco_pass[np.random.randint(len(marco_pass))]

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import random 

In [ ]:
# A popular sentence embedding model used for search. It outputs unit norm vectors 
# and the  model is trained with contrastive loss. 
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
random.shuffle(marco_pass)

In [ ]:
vectors = model.encode(marco_pass[:10_000])

In [ ]:
dot_products = vectors @ vectors.T
dot_products = dot_products[np.triu(dot_products, k=1) != 0]
angles = np.arccos(dot_products[dot_products < 1])*(180/np.pi)

In [ ]:
# plt.hist(np.random.normal(0,1/np.sqrt(vectors.shape[-1]),size=len(dot_products)),range=(-0.2,0.2),bins=100)
# plt.hist(dot_products,histtype="step",range=(-0.2,0.2),bins=100)

In [ ]:
plt.figure(figsize=(6,4),dpi=150)
plt.hist(angles,bins=100,range=(70,110))
plt.title("Most vectors are almost orthogonal!")
plt.xlabel("Angle in degrees")
plt.show()

We find that vectors are quite orthogonal to each other -- which is the expected behaviour of random vectors in large dimensions. The true distribution of dot products of random vectors in $d$ dimensions is given by a Beta distrbution and the above does not follow it, but nevertheless it is not as anisotropic as people make it out to be.  

In fact, since random vectors in large dimensions can be modelled as $\mathcal N\left(0,\frac{1}{\sqrt{d}}I_d\right)$. Let us check if our vectors look like it?

In [ ]:
d = vectors.shape[-1]
idx = np.random.randint(d)
plt.figure(figsize=(6,4),dpi=150)
plt.hist(vectors[:,idx], range=(-0.2,0.2),bins=100)
plt.hist(np.random.normal(0,1/np.sqrt(d),size=len(vectors)),range=(-0.2,0.2),bins=100,histtype="step")
plt.title("The vectors seem like istropic Gaussian")
plt.show()

We can also create a Q-Q plot to check Gaussianity.

In [ ]:
# Create a Q-Q plot against the normal distribution
idx = np.random.randint(d)
fig, ax = plt.subplots(figsize=(6, 6),dpi=100)
stats.probplot(vectors[:,idx], dist='norm', plot=ax)
ax.set_title('Q-Q Plot')
ax.set_xlabel('Theoretical Quantiles')
ax.set_ylabel('Sample Quantiles')

plt.show()

It is difficult to test whether a distribution is Gaussian. But using quantile plots and the histograms above we find that there is non zero evidence for the vectors to be isotropic. Why is this the case? What magic of contrastive loss training achieves this? This brings us back to the question we started with at first -- $k$th roots of unity on $S^1$, except now we are in $S^d$. 

Hopefully you have found enough interesting questions/ideas here to make something useful out of it!
